# Char RNN - Prototype

The main objective of this project is to set up a workflow for defining, training and optimizing neural networks.

We are going to predict next characters from sample text, similar to [char-rnn](https://github.com/karpathy/char-rnn).

We create a notebook which will serve as prototyping our model.

## Load and prepare data

We first download the [The Count of Monte Cristo](https://www.gutenberg.org/ebooks/1184) from Project Gutenberg.

In [2]:
from pathlib import Path
import requests
import zipfile

PATH_DATA = Path("data")
FILENAME_DATA = Path("monte_cristo.txt")
URL_DATA = "https://www.gutenberg.org/files/1184/1184-0.txt"

# Download dataset
PATH_DATA.mkdir(exist_ok = True)
PATH_DATAFILE = PATH_DATA / FILENAME_DATA
if not (PATH_DATAFILE).exists():
    r = requests.get(URL_DATA)
    PATH_DATAFILE.open("wb").write(r.content)

We read the entire text and keep only the interesting lines by removing titles, bibliography…

In [3]:
with open(PATH_DATAFILE, 'r', encoding="utf8") as f:
    lines = f.readlines()
    # Remove start and end of file (not interesting data)
    lines = lines[319:60662]
    chars = ''.join(lines)
            
# Test code

print("Sample text:\n")
print(chars[:276])

Sample text:

On the 24th of February, 1815, the look-out at Notre-Dame de la Garde
signalled the three-master, the Pharaon from Smyrna, Trieste, and
Naples.

As usual, a pilot put off immediately, and rounding the Château d’If,
got on board the vessel between Cape Morgiou and Rion island.


In [4]:
# Test code

print("Total number of chars:", len(chars))
print("Unique chars:", len(set(chars)))

Total number of chars: 2617219
Unique chars: 99


We then create a dictionary for mapping between chars and numbers.

In [ ]:
# Adapted from https://github.com/pytorch/examples/blob/master/word_language_model/data.py

class Dictionary(object):
    def __init__(self):
        self.char2idx = {}
        self.idx2char = []

    def add_char(self, char):
        if char not in self.char2idx:
            self.idx2char.append(char)
            self.char2idx[char] = len(self.idx2char) - 1
        return self.char2idx[char]

    def __len__(self):
        return len(self.idx2char)

We finally convert our data from char to token.

In [ ]:
import torch

data_dictionary = Dictionary()
tensor_data = torch.LongTensor(len(chars))

for i, c in enumerate(chars):
    tensor_data[i] = data_dictionary.add_char(c)
    
n_elements = len(data_dictionary)
    
# Transform into one-hot (source: https://discuss.pytorch.org/t/convert-int-into-one-hot-format/507/29)
input_data = torch.zeros(len(tensor_data), n_elements).scatter_(1, tensor_data.unsqueeze(-1), 1)
label_data = tensor_data  # we don't need labels in one-hot format

In [ ]:
# Test code

print("Sample values:")
print('\n'.join('{1} ({0})'.format(idx, data_dictionary.idx2char[idx.argmax()]) for idx in input_data[506:511]))

Sample values:
i (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]))
g (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]))
g (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

Finally, we split the data between test and validation sets.

In [ ]:
split = round(0.95 * len(tensor_data))
train_data, train_label = input_data[:split], label_data[1:split+1]
valid_data, valid_label = input_data[split:-2], label_data[split+1:]

Let's create a class to handle our training data in batch.

In [ ]:
class TrainingData():
    
    def __init__(self, train_data, train_label, sequence_per_batch = 64, char_per_sequence = 128):
        
        self.train_data = train_data
        self.train_label = train_label
        self.sequence_per_batch = sequence_per_batch
        self.char_per_sequence = char_per_sequence
        self.length = len(train_data)
        
        # We start reading the text at even sections based on number of sequence per batch
        self.batch_idx = range(0, self.length, self.length // sequence_per_batch)
        self.batch_idx = self.batch_idx[:sequence_per_batch]
        assert len(self.batch_idx) == sequence_per_batch, '{} batches expected vs {} actual'.format(sequence_per_batch,
                                                                                                    len(self.batch_idx))
        
    def next_batch(self):
        
        # loop to the start if we reached the end of text
        self.batch_idx = list(idx if idx + self.char_per_sequence < self.length else 0 for idx in self.batch_idx)
        
        # Extract sequences
        sequences_input = tuple(self.train_data[idx:idx+self.char_per_sequence] for idx in self.batch_idx)
        sequences_label = tuple(self.train_label[idx:idx+self.char_per_sequence] for idx in self.batch_idx)
        
        # Move next idx
        self.batch_idx = (idx + self.char_per_sequence for idx in self.batch_idx)
        
        # Concatenate tensors
        return torch.stack(sequences_input, dim=1), torch.stack(sequences_label, dim=1)

## Create and train a neural network

We will create & optimize different variants of following architecture.

![architecture](img/architecture.png)

We want to optimize the RNN module and will create our architecture so we can easily test different variants by choosing:

* RNN, LSTM or GRU modules

* number of features for hidden states

* number of layers

* dropout between each layer

We can decide to optimize other parameters such as the loss function, optimization algorithm…

In [ ]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_size, batch_size, rnn_module = "RNN", hidden_size = 64, num_layers = 1, dropout = 0):
        super(Model, self).__init__()
        self.input_size = input_size
        self.rnn_module = rnn_module
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if rnn_module == "RNN":
            self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        elif rnn_module == "LSTM":
            self.rnn = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        elif rnn_module == "GRU":
            self.rnn = nn.GRU(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
            
        self.output = nn.Linear(hidden_size, input_size)

    def forward(self, input, hidden):
        output = input.view(1, -1, self.input_size)
        output, hidden = self.rnn(output, hidden)
        output = self.output(output[0])
        return output, hidden

    def initHidden(self, batch_size):
        # initialize hidden state with a sequence of word tokens
        if self.rnn_module == "LSTM":
            return torch.zeros(self.num_layers, batch_size, self.hidden_size), torch.zeros(
                self.num_layers, batch_size, self.hidden_size)
        else:
            return torch.zeros(self.num_layers, batch_size, self.hidden_size)

We now need to define a loss and optimizer.

In [ ]:
import torch.optim as optim

loss_function = nn.CrossEntropyLoss()
optimizer_function = optim.Adam

We can now train the model.

In [ ]:
from tqdm import tnrange
from numpy import random

# Define hyper-parameters
rnn_module = "GRU"
hidden_size = 128
num_layers = 3
dropout = 0.1
epochs = 100
batches_per_epoch = 300
sequence_per_batch = 8
char_per_sequence = 150

# Build the NN
model = Model(len(data_dictionary), sequence_per_batch, rnn_module, hidden_size, num_layers, dropout)
hidden = model.initHidden(sequence_per_batch)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")   # for some reason, it goes much faster in my experiments
train_data = train_data.to(device)
train_label = train_label.to(device)
valid_data = valid_data.to(device)
valid_label = valid_label.to(device)
model.to(device)
hidden = hidden.to(device)

# Define optimizer
optimizer = optimizer_function(model.parameters())

# Load data
training_data = TrainingData(train_data, train_label, sequence_per_batch, char_per_sequence)

valid_length = len(valid_data)

for epoch in tnrange(epochs):
    train_loss = 0   # training loss
    valid_loss = 0   # validation loss
    
    # Training of one epoch
    model.train()
    for i in tnrange(batches_per_epoch):
        
        # Get a batch of sequences
        input_vals, label_vals = training_data.next_batch()

        # Detach hidden layer and reset gradients
        if rnn_module == "LSTM":
            tuple(h.detach_() for h in hidden)
        else:
            hidden.detach_()
        optimizer.zero_grad()
        
        # Forward pass and calculate loss
        loss_sequence = torch.zeros(1, device=device)
        for (input_val, label_val) in zip(input_vals, label_vals):
            output, hidden = model(input_val, hidden)
            loss = loss_function(output, label_val.view(-1))
            loss_sequence += loss
            
        # Backward propagation and weight update
        loss_sequence.backward()
        optimizer.step()
        
        train_loss += loss_sequence.item() / batches_per_epoch / char_per_sequence
        
    # Calculate validation loss
    with torch.no_grad():
        model.eval()
        hidden_valid = model.initHidden(1).to(device)
        for i in range(valid_length-1):
            input_val = valid_data[i]
            label_val = valid_label[i].view(1)
            output, hidden_valid = model(input_val, hidden_valid)
            loss = loss_function(output, label_val)
            valid_loss += loss.item() / (valid_length - 1)
        
    print("Epoch {} - Training loss {} - Validation loss {}".format(epoch+1, train_loss, valid_loss))

Epoch 1 - Training loss 3.1934996656629773 - Validation loss 3.144932621842185


Epoch 2 - Training loss 2.8241447692871127 - Validation loss 2.4369557873105694


Epoch 3 - Training loss 2.219004919433594 - Validation loss 2.0687018499885386


Epoch 4 - Training loss 1.979423703342013 - Validation loss 1.8893669783129634


Epoch 5 - Training loss 1.8262373640272354 - Validation loss 1.7676303601661845


...

Epoch 28 - Training loss 1.3383069573296447 - Validation loss 1.310190858221341


Epoch 29 - Training loss 1.3629359188503698 - Validation loss 1.3080112175446816


Epoch 30 - Training loss 1.3396507527669272 - Validation loss 1.3078746282498113


Epoch 31 - Training loss 1.3195872148301877 - Validation loss 1.296608580798723


Epoch 32 - Training loss 1.3151345499674465 - Validation loss 1.2977992571244428


We make sure to save the trained model.

In [ ]:
# Save model
PATH_MODEL = "model.pt"
torch.save(model, PATH_MODEL)

# Load model
model = torch.load(PATH_MODEL)
model.eval()

## Test the model

We finally test the model by predicting a few characters.

In [ ]:
from numpy import random

with torch.no_grad():
    
    # Define a sequence of characters to initialize the hidden states
    init_chars = "The "

    init_data = torch.LongTensor(len(init_chars))
    for i, c in enumerate(init_chars):
        init_data[i] = data_dictionary.char2idx[c]

    # Transform into one-hot
    init_data = torch.zeros(len(init_data), len(data_dictionary)).scatter_(1, init_data.unsqueeze(-1), 1)

    # Initialize hidden layer and feed sequence of characters to the model
    hidden = model.initHidden(1)
    for init_char in init_data:
        output, hidden = model(init_char, hidden)

    # Predict next characters one at a time
    number_chars = 500
    chars = init_chars
    for _ in range(number_chars):

        # Calculate probability distribution of outputs with a temperature of 0.5
        prob = nn.Softmax(1)(output/0.5).squeeze().numpy()

        # Sample from outputs
        output_idx = random.choice(len(prob), p = prob)

        # Extract predicted char
        predicted_char = data_dictionary.idx2char[output_idx]
        chars += predicted_char

        # Transform predicted char into one-hot vector
        output_idx = torch.LongTensor([[output_idx]])
        next_input = torch.zeros(len(output_idx), len(data_dictionary)).scatter_(1, output_idx, 1)

        # Feed into NN to predict next char
        output, hidden = model(next_input, hidden)

    # Print predicted sequence
    print("Initializing sequence:", init_chars)
    print("Predicted sequence:", chars)    

So it seems to be working and we now want to do some optimization.

## Optimization

At this point we would want to fine-tune by implementing:

* monitoring features (plot graph in real time)

* comparison of past experiments

However we would be wasting our time. [Weights & Biases](https://www.wandb.com/) will do that for us quicker and will provide additional insight tools.

We have now completed our prototype and it is time to move our code to a separate Python file (or multiple for organized people). This will help in cleaning our code and running multiple experiments at the same time (locally or remotely).

Refer to my tutorial for more details.